In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

from dotenv import load_dotenv
import sys
import os
import datetime
# Добавляем корневую директорию проекта в sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

from consultant.tools.basic import normalize_path

load_dotenv()

spark = (SparkSession.builder
         .appName("cbr_currency_etl")
         .getOrCreate())

spark

In [9]:
from consultant.extractors.base import get_cbr_currency_rate_json


currency_list = list(get_cbr_currency_rate_json()['Valute'].values())

raw_currency = spark.createDataFrame(currency_list)
raw_currency.show(100, truncate=False)

/home/b.kustov/.local/lib/python3.8/site-packages/pyspark/sql/session.py:381: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+--------+-------+----------------------------------------+-------+-------+--------+--------+
|CharCode|ID     |Name                                    |Nominal|NumCode|Previous|Value   |
+--------+-------+----------------------------------------+-------+-------+--------+--------+
|AUD     |R01010 |Австралийский доллар                    |1      |036    |65.8247 |66.7047 |
|AZN     |R01020A|Азербайджанский манат                   |1      |944    |60.7475 |61.1471 |
|GBP     |R01035 |Фунт стерлингов Соединенного королевства|1      |826    |131.5462|132.4739|
|AMD     |R01060 |Армянских драмов                        |100    |051    |26.1876 |26.3625 |
|BYN     |R01090B|Белорусский рубль                       |1      |933    |29.885  |29.8278 |
|BGN     |R01100 |Болгарский лев                          |1      |975    |55.584  |55.8436 |
|BRL     |R01115 |Бразильский реал                        |1      |986    |17.0622 |17.2339 |
|HUF     |R01135 |Венгерских форинтов                     |1

In [10]:
prepared_currency = raw_currency \
    .withColumn('VunitRate', col('Value') / col('Nominal')) \
    .withColumn('ValCursDate', lit(datetime.date.today())) \
    .select(['CharCode', 'Nominal', 'Value', 'VunitRate', 'ValCursDate'])
prepared_currency.show(100, truncate=False)

+--------+-------+--------+--------------------+-----------+
|CharCode|Nominal|Value   |VunitRate           |ValCursDate|
+--------+-------+--------+--------------------+-----------+
|AUD     |1      |66.7047 |66.7047             |2024-12-13 |
|AZN     |1      |61.1471 |61.1471             |2024-12-13 |
|GBP     |1      |132.4739|132.4739            |2024-12-13 |
|AMD     |100    |26.3625 |0.263625            |2024-12-13 |
|BYN     |1      |29.8278 |29.8278             |2024-12-13 |
|BGN     |1      |55.8436 |55.8436             |2024-12-13 |
|BRL     |1      |17.2339 |17.2339             |2024-12-13 |
|HUF     |100    |26.6573 |0.266573            |2024-12-13 |
|VND     |10000  |42.8501 |0.00428501          |2024-12-13 |
|HKD     |1      |13.3922 |13.3922             |2024-12-13 |
|GEL     |1      |36.7016 |36.7016             |2024-12-13 |
|DKK     |1      |14.6458 |14.6458             |2024-12-13 |
|AED     |1      |28.305  |28.305              |2024-12-13 |
|USD     |1      |103.95

In [27]:
# Writing

from consultant.tools.basic import get_jdbc_url_for_gp, get_properies_for_gp

# import importlib
# import consultant.tools.basic as module

# importlib.reload(module)

# print(get_properies_for_gp())
# print(get_jdbc_url_for_gp())

prepared_currency.write.mode('overwrite').jdbc(
    get_jdbc_url_for_gp(),
    'b.kustov.currency_rate',
    get_properies_for_gp()
)

Py4JError: An error occurred while calling o204.mode. Trace:
py4j.Py4JException: Method mode([class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:750)



In [12]:
spark.stop()